In [1]:
mod_input=True
mod_topo=True
leak_rate = 0.2
gradient= 0.00075# 0.0005
gain = 1.75
radical=False
radical_bis=False
grad2=1.25
breadth=600
expon=3

res_seed = 4

# baseline with slow leak rate

In [2]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pylab
import statsmodels.api as sm

import seaborn as sns

import csv

import scipy
from scipy import stats
from scipy.ndimage import gaussian_filter1d

import random


In [3]:
#import cupy as cp
from easyesn.optimizers import GradientOptimizer
from easyesn import PredictionESN
from easyesn.optimizers import GridSearchOptimizer
from easyesn import helper as hlp
import numpy as np
#from wikipedia2vec import Wikipedia2Vec




vectorDim = 100

numNode = 1000

#inputDataTraining = np.load('./trainingData_averaging/inputDataTraining_4k_average.npy')
#outputDataTraining = np.load('./trainingData_averaging/outputDataTraining_4k_average.npy')
#print(len(inputDataTraining))
#print(len(outputDataTraining))

Using Numpy backend.


In [4]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [ ]:
#reservoirStatesBuffer[:,:resn]

#in_small = inputDataTraining[:10000]
#out_small = outputDataTraining[:10000]


#np.save('in_small.npy', in_small)
#np.save('out_small.npy', out_small)

in_small = np.load('C:/Users/PeterDell/Google Drive/GoogleWIP/People/Uchida/PaperMethods/UchidaPrograms/in_small.npy')
out_small = np.load('C:/Users/PeterDell/Google Drive/GoogleWIP/People/Uchida/PaperMethods/UchidaPrograms/out_small.npy')

print(len(in_small))
print(len(out_small))

In [ ]:
wiki2vec = Wikipedia2Vec.load('C:/Users/PeterDell/Google Drive/GoogleWIP/People/Uchida/PaperMethods/UchidaPrograms/enwiki_20180420_100d.pkl')



In [ ]:
temp_W=np.zeros([1000,1000])

In [7]:
inputDataTesting = np.load("synthetic_narrative2.npy")
ground_truth_sums = np.load("synthetic_narrative2-gt.npy")

In [8]:
print(inputDataTesting.shape)

(1740, 100)


In [9]:
rand_temp = random.randint(0,10)
print(rand_temp)

3


In [59]:
num_bins_list = [5, 10, 20, 50]
num_bins_list = [5, 10, 20, 50, 100, 200, 500]
total_length = inputDataTesting.shape[0]
#total_length = 100
bin_set = np.zeros([len(num_bins_list),500])
sorted_indexes=np.zeros([bin_set.shape[0],total_length])
bin_set.shape

(7, 500)

In [60]:
#create the set of different segmentation into bins in bin_set[7,500]

bin_index = -1

for num_bins in num_bins_list:
    bin_index = bin_index + 1
    searching = True
    while searching:
        bin_sizes=np.zeros([num_bins])
        mean_bin_size = int(total_length/num_bins)
        for i in range(num_bins):
            bin_sizes[i] = int(mean_bin_size/2 + random.randint(0,int(total_length/num_bins)) + 0.5)
            #bin_sizes[i] = int(random.randint(1,int(2*total_length/num_bins)))
        factor = int((total_length-sum(bin_sizes))/num_bins)
        bin_sizes = bin_sizes + factor
        if sum(bin_sizes) == total_length:
            bin_set[bin_index,:num_bins] = bin_sizes
            print("initial calculated bin sizes for ", num_bins, "bins\n", bin_sizes)
            print("stored calculated bin sizes in bin_set\n", bin_set[bin_index,:num_bins])
            print(sum(bin_sizes), sum(bin_set[bin_index]), total_length, "\n")
            
            searching = False
        else:
            searching = True



initial calculated bin sizes for  5 bins
 [517. 239. 326. 223. 435.]
stored calculated bin sizes in bin_set
 [517. 239. 326. 223. 435.]
1740.0 1740.0 1740 

initial calculated bin sizes for  10 bins
 [143. 139. 156. 244. 226. 185. 253. 129.  94. 171.]
stored calculated bin sizes in bin_set
 [143. 139. 156. 244. 226. 185. 253. 129.  94. 171.]
1740.0 1740.0 1740 

initial calculated bin sizes for  20 bins
 [126.  62.  85. 128.  73. 105. 110.  94.  60.  68.  78.  52. 124.  85.
  49.  97. 126.  91.  72.  55.]
stored calculated bin sizes in bin_set
 [126.  62.  85. 128.  73. 105. 110.  94.  60.  68.  78.  52. 124.  85.
  49.  97. 126.  91.  72.  55.]
1740.0 1740.0 1740 

initial calculated bin sizes for  50 bins
 [27. 35. 28. 24. 43. 21. 46. 22. 47. 33. 31. 35. 18. 20. 31. 43. 41. 26.
 39. 39. 34. 30. 49. 28. 41. 23. 26. 30. 38. 42. 33. 49. 44. 39. 18. 39.
 28. 44. 46. 28. 30. 49. 47. 47. 17. 22. 42. 45. 40. 43.]
stored calculated bin sizes in bin_set
 [27. 35. 28. 24. 43. 21. 46. 22. 47. 3

In [61]:
#now for each of the binnings, do the segmentation, shuffling, and creation of sorted
#indexes in sorted_indexes[7,1740]
bin_set[1,:10]

array([143., 139., 156., 244., 226., 185., 253., 129.,  94., 171.])

In [62]:
#create a list of indices into 
index_list = []
#for i in range(inputDataTesting.shape[0]):
for i in range(total_length):
    index_list.append(i)

In [63]:
print(index_list)
temp = index_list[20:30]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [64]:
# now split that list into a list of lists
last_value = 0
list_of_lists = []
for chunk_size in bin_set[1,:10]:
    temp_list = index_list[int(last_value):int(last_value+chunk_size)]
    print(temp_list)
    last_value = last_value+chunk_size
    list_of_lists.append(temp_list)
random.shuffle(list_of_lists)
print(list_of_lists)    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142]
[143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221

In [65]:
flat_list = [item for sublist in list_of_lists for item in sublist]

In [66]:
print(flat_list)

[908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089,

In [71]:
# we now have bin_set[7,500] with the [5, 10, 20, 50, 100, 200, 500] bins
#now for each of the binnings, do the segmentation, shuffling, and creation of sorted
#indexes in sorted_indexes[7,1740]

for binning in range(bin_set.shape[0]):
    
    #create a list of indices into the embeddings
    index_list = []
    #for i in range(inputDataTesting.shape[0]):
    for i in range(total_length):
        index_list.append(i)
        
    # now split that list into a list of lists
    last_value = 0
    list_of_lists = []
    for chunk_size in bin_set[binning,:num_bins_list[binning]]:
        temp_list = index_list[int(last_value):int(last_value+chunk_size)]
    
        last_value = last_value+chunk_size
        list_of_lists.append(temp_list)
    #print(list_of_lists)
    print(bin_set[binning,:num_bins_list[binning]])
    random.shuffle(list_of_lists)

    print(list_of_lists,"\n\n")
    
    flat_list = [item for sublist in list_of_lists for item in sublist]
    print("flat_list",flat_list,"\n\n")
    
    
    #put the flat_list into sorted_indexes[bin_set.shape[0],total_length]
    for i in range(total_length):
        sorted_indexes[binning,i] = flat_list[i]

[517. 239. 326. 223. 435.]
[[1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466,

In [72]:
np.save('sorted_indexes.npy', sorted_indexes)

In [48]:
flat_list

[]

In [ ]:
#to shuffle or notto suffle!
do_shuffle = True
if do_shuffle:
    random.shuffle(inputDataTesting)
    random.shuffle(inputDataTesting)
    random.shuffle(inputDataTesting)
        
for res_seed in range(40):
# set the seed
    np.random.seed(res_seed)

    # set up the reservoir
    # leakingRate=0.2 changing too fast, mutliple with 0.05; changed
    esn = PredictionESN(n_input=vectorDim, n_output=vectorDim, n_reservoir=numNode, leakingRate=leak_rate, regressionParameters=[1e-2], solver="lsqr", feedback=False)

    # train the reervoir with a new seed
    esn.fit(in_small, out_small, transientTime="Auto", verbose=1)
    


    # modify the reservoir topology for input
    if mod_input:
        for i in range(700):
            for j in range(101):
                esn._WInput[i+300,j] = 0
        esn._WInput=esn._WInput*5

    #radical _W modification
    if mod_topo:
        for i in range(1000):
            for j in range(1000):
                temp_W[i,j] = esn._W[i,j]
        for i in range(1000):
            for j in range(1000):
                esn._W[i,j]=0
        for i in range(esn._W.shape[0]):
            for j in range(esn._W.shape[1]):
                if abs(i-j) <= breadth:
                    esn._W[i,j] = ((breadth-abs(i-j))/breadth)**expon*temp_W[i,j]* (1 + i * gradient)*gain
    if radical:
        for i in range(100,200):
            for j in range(800,900):
                esn._W[j,i] = temp_W[j,i]*grad2

    if radical_bis:
        for i in range(100,200):
            for j in range(800,900):
                esn._W[i,j] = temp_W[j,i]*grad2


    #run the reservoir
    reservoirStatesBuffer = np.empty((0,numNode))
    prediction,reservoirStatesBuffer  = esn.predict(inputDataTesting)
    reservoirStatesBuffer = reservoirStatesBuffer.T
    print(reservoirStatesBuffer.shape)
    reservoirStatesBuffer = reservoirStatesBuffer[:,101:]
    print(reservoirStatesBuffer.shape)
    print(prediction.shape)

    # set the similarity matrix
    sim_matrix1 = np.empty([len(reservoirStatesBuffer),len(reservoirStatesBuffer)])

    print(len(reservoirStatesBuffer[1]))

    reservoirStatesBuffer1 = reservoirStatesBuffer

    if do_shuffle:
        file_string = "synth_narr_reservoirs/structured_synthetic_scrambled/data/struct_synthet_scrambled_%d.npy" % (res_seed)
    else:
        file_string = "synth_narr_reservoirs/structured_synthetic_intact/data/struct_synthet_intact_%d.npy" % (res_seed)
    print("saving file", file_string)
    np.save(file_string,reservoirStatesBuffer)
    
    #plot figure to verigy
    resn = 50
    begin=0
    end=2200
    rbegin = 900
    rend = 910
    fig= plt.figure(figsize=(10,12))

    plt.subplot(611)
    plt.plot( reservoirStatesBuffer1[begin:end,100:105])
    plt.title('Hierarchical 100:105')

    plt.subplot(612)
    plt.plot( reservoirStatesBuffer1[begin:end,300:305])
    plt.title('Hierarchical 300:305')

    plt.subplot(613)
    plt.plot( reservoirStatesBuffer1[begin:end,500:505])
    plt.title('Hierarchical 500:505')

    plt.subplot(614)
    plt.plot( reservoirStatesBuffer1[begin:end,700:705])
    plt.title('Hierarchical 700:705')

    plt.subplot(615)
    plt.plot( reservoirStatesBuffer1[begin:end,900:905])
    plt.title('Hierarchical 900:905')

    plt.subplot(616)
    plt.plot( inputDataTesting[begin:end,:])
    plt.title('Input')

    fig.tight_layout()

    if do_shuffle:
        plt.savefig('synth_narr_reservoirs/structured_synthetic_scrambled/figures/struct_synthet_scrambled_%d.png' % (res_seed))
    else:
        plt.savefig('synth_narr_reservoirs/structured_synthetic_intact/figures/struct_synthet_intact_%d.png' % (res_seed))
        
    numNode=1000
    for j in range(numNode):
        mean = reservoirStatesBuffer[:,j].mean()
        if abs(mean) > 0.001:
            reservoirStatesBuffer[:,j] = reservoirStatesBuffer[:,j]/mean
        
    fig= plt.figure(figsize=(10,8))
    sns.heatmap(np.corrcoef(reservoirStatesBuffer[:,:]),cmap='viridis')#sns.heatmap(sim_matrix, square=True)
    for i in ground_truth_sums:
        plt.plot(i, i,marker="o", markersize=8, markeredgecolor="red", markerfacecolor="red")

    if do_shuffle:
        plt.savefig('synth_narr_reservoirs/structured_synthetic_scrambled/figures/struct_heat_scrambled_%d.png' % (res_seed))
    else:
        plt.savefig('synth_narr_reservoirs/structured_synthetic_intact/figures/struct_heat_intact_%d.png' % (res_seed))
        
        
        
    plt.close('all')    

In [ ]:
#np.save('reservoir_sherlock_hierarchy_seed4.npy',reservoirStatesBuffer)

In [ ]:
plt.imshow(esn._W,cmap='seismic')
plt.colorbar()

In [ ]:
plt.imshow(esn._WInput,cmap='seismic' )

In [ ]:
resn = 50
begin=0
end=680
rbegin = 900
rend = 910
fig= plt.figure(figsize=(10,12))

plt.subplot(511)
plt.plot( reservoirStatesBuffer1[begin:end,100:105])
plt.title('Hierarchical 100:105')

plt.subplot(512)
plt.plot( reservoirStatesBuffer1[begin:end,300:305])
plt.title('Hierarchical 300:305')

plt.subplot(513)
plt.plot( reservoirStatesBuffer1[begin:end,500:505])
plt.title('Hierarchical 500:505')

plt.subplot(514)
plt.plot( reservoirStatesBuffer1[begin:end,700:705])
plt.title('Hierarchical 700:705')

plt.subplot(515)
plt.plot( reservoirStatesBuffer1[begin:end,900:905])
plt.title('Hierarchical 900:905')


fig.tight_layout()

#plt.savefig('temporal_hierarchy.png')

In [ ]:
print(reservoirStatesBuffer1.shape)

In [ ]:
plt.plot( sm.tsa.stattools.ccf(reservoirStatesBuffer1[:,300], reservoirStatesBuffer1[:,350])[:20],linewidth=2 )
plt.plot( sm.tsa.stattools.ccf(reservoirStatesBuffer1[:,300], reservoirStatesBuffer1[:,450])[:20],linewidth=4 )
plt.plot( sm.tsa.stattools.ccf(reservoirStatesBuffer1[:,300], reservoirStatesBuffer1[:,550])[:20],linewidth=8 )

In [ ]:
reservoirStatesBuffer1.shape

In [ ]:
areas = 20
factor = int(numNode/areas)
roi_data = np.zeros([areas,reservoirStatesBuffer1.shape[0]])
print(roi_data.shape)
print(factor)

In [ ]:
for i in range(areas): # create 10 sub areas
    for j in range (reservoirStatesBuffer1.shape[0]): #for each timestep
        roi_data[i,j] = np.mean(reservoirStatesBuffer1[j,i*factor:(i+1)*factor])

In [ ]:
reservoirStatesBuffer1[j,i*factor:(i+1)*factor]

In [ ]:
plt.plot(roi_data.T)

plt.plot()

In [ ]:
plt.plot(roi_data[8:15,50:150].T)

plt.plot()

In [ ]:
source = 6
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[8,50:150].T)[:10],linewidth=1 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[10,50:150].T)[:10],linewidth=2 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[12,50:150].T)[:10],linewidth=3 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[14,50:150].T)[:10],linewidth=4 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[16,50:150].T)[:10],linewidth=5 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[18,50:150].T)[:10],linewidth=6 )
#plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[19,50:150].T)[:10],linewidth=7 )

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1, sharex=True)
ax1.xcorr(reservoirStatesBuffer1[700,50:150].T, reservoirStatesBuffer1[800,50:150].T, usevlines=True, maxlags=50, normed=True)
ax1.grid(True)

ax2.acorr(reservoirStatesBuffer1[16,50:150].T, usevlines=True, normed=True, maxlags=50)
ax2.grid(True)

plt.show()

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1, sharex=True)
ax1.xcorr(roi_data[7,50:150].T, roi_data[6,50:150].T, usevlines=False, maxlags=50, normed=True)
ax1.grid(True)

ax2.acorr(roi_data[16,50:150].T, usevlines=False, normed=True, maxlags=50)
ax2.grid(True)

plt.show()






In [ ]:
wiki2vec.get_word_vector("aezr")

In [ ]:
word = "qsdf"
if '<Word sangane>' in wiki2vec.dictionary:
    print("great")

In [ ]:
for token in wiki2vec.dictionary:
    print(token)

In [ ]:
if wiki2vec.get_word('qsdfqsdf'):
    print("great")